In [2]:
import pandas as pd

In [3]:
# loading the dataset
pd.read_csv("data/Esophageal_Dataset.csv")

,Unnamed: 0,patient_barcode,tissue_source_site,patient_id,bcr_patient_uuid,informed_consent_verified,icd_o_3_site,icd_o_3_histology,icd_10,tissue_prospective_collection_indicator,...,primary_pathology_lymph_node_examined_count,primary_pathology_number_of_lymphnodes_positive_by_he,primary_pathology_number_of_lymphnodes_positive_by_ihc,primary_pathology_planned_surgery_status,primary_pathology_treatment_prior_to_surgery,primary_pathology_residual_tumor,primary_pathology_karnofsky_performance_score,primary_pathology_eastern_cancer_oncology_group,primary_pathology_radiation_therapy,primary_pathology_postoperative_rx_tx
0,0,TCGA-2H-A9GF,2H,A9GF,0500F1A6-A528-43F3-B035-12D3B7C99C0F,YES,C15.5,8140/3,C15.5,NO,...,8.0,7.0,0.0,NaN,NaN,R1,NaN,NaN,NO,NO
1,1,TCGA-2H-A9GG,2H,A9GG,70084008-697D-442D-8F74-C12F8F598570,YES,C15.5,8140/3,C15.5,NO,...,19.0,4.0,0.0,NaN,NaN,R1,NaN,NaN,NO,NO
2,2,TCGA-2H-A9GH,2H,A9GH,606DC5B8-7625-42A6-A936-504EF25623A4,YES,C15.5,8140/3,C15.5,NO,...,30.0,1.0,0.0,NaN,NaN,R0,NaN,NaN,NO,NO
3,3,TCGA-2H-A9GI,2H,A9GI,CEAF98F8-517E-457A-BF29-ACFE22893D49,YES,C15.5,8140/3,C15.5,NO,...,8.0,4.0,0.0,NaN,NaN,R0,NaN,NaN,NO,NO
4,4,TCGA-2H-A9GJ,2H,A9GJ,EE47CD59-C8D8-4B1E-96DB-91C679E4106F,YES,C15.5,8140/3,C15.5,NO,...,19.0,0.0,0.0,NaN,NaN,R0,NaN,NaN,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,3995,677b83fe-4dbd-410e-aea8-62d68a80f652,Z6,A8JD,B997D486-8C71-4926-9DF1-D2AFDAF2785F,YES,C15.9,8070/3,C15.9,YES,...,19.0,0.0,0.0,NaN,NaN,R0,90.0,2.0,NaN,NaN
3981,3996,a4faa9ff-b9a1-4a4c-9929-bb0154030067,Z6,A8JE,1116DDAE-76B0-48D6-869C-BE65D8B09D35,YES,C15.3,8070/3,C15.3,YES,...,9.0,1.0,0.0,Surgery already performed,No Treatment,RX,80.0,2.0,NaN,NaN
3982,3997,6bdc4fc8-8d76-4886-bf13-e282f4b1476b,Z6,A9VB,1C2837D4-BD2C-46F7-9E22-683B82A16350,YES,C15.5,8070/3,C15.5,YES,...,NaN,NaN,NaN,NaN,NaN,R0,NaN,NaN,NO,NO
3983,3998,d61e44e1-06ed-4849-85d7-3b360dd03721,Z6,AAPN,2659BB79-95F9-4476-A516-F3800536422D,YES,C15.5,8070/3,C15.5,YES,...,11.0,0.0,NaN,NaN,NaN,R0,NaN,NaN,NO,NO
